In [5]:
import os
from mocap_visualizer.generate_views import (    # organize this
    get_output_dir,
    prepare_videos
)
from mocap_visualizer.animated_phase_analysis import generate_all_animations
from mocap_visualizer.animated_merged_phase_analysis import animate_merged_phase_analysis
from mocap_visualizer.kinematic_visualizer import visualize_joint_position
from mocap_visualizer.video_layout import combine_views

In [4]:
# Configuration
filename = "BKO_E1_D2_03_Suku"
bvh_file = filename + "_T"
start_time = 122.0
end_time = 125.0

# path to onsets and cycles csv files
cycles_csv_path = f"data/virtual_cycles/{filename}_C.csv"
onsets_csv_path = f"data/drum_onsets/{filename}.csv"
# dance_csv_path = f"data/dance_onsets/{filename}_T_dance_onsets.csv"

# prepare output directory
output_dir = get_output_dir(bvh_file, start_time, end_time)
output_fps = 24

### Generate Separate Drum distribution plots

In [ ]:
print("\nGenerating distribution plot videos...")

# generate animated distribution plots
generate_all_animations(
    filename, start_time, end_time,
    cycles_csv_path, onsets_csv_path,
    save_dir=output_dir,
    figsize=(10, 3), dpi=200
)

### Generate merged distribution plot (DunDun, J1, J2)

In [ ]:
merged_save_path = "drum_combined.mp4"
animate_merged_phase_analysis(
    filename, start_time, end_time,
    cycles_csv_path, onsets_csv_path,
    figsize=(10, 4), dpi=200,
    save_path=merged_save_path,
    save_dir=output_dir
)

### Generate Skeleton Videos + trimmed video_mix + audio

In [ ]:
video_size = (640, 360)
# views_to_generate = ['front', 'right']

output_dir, view_videos = prepare_videos(
    filename=bvh_file,
    start_time=start_time,
    end_time=end_time,
    video_path= f"{filename}_pre_R_Mix.mp4",
    video_size=video_size,
    fps=output_fps
)

### Generate animated kinematic plots

In [7]:
joint_name = "LeftAnkle"  
axis = 'y'

In [ ]:
# Generate joint position visualization

joint_video = os.path.join(output_dir, f"{joint_name}_{axis}_position.mp4")
if not os.path.exists(joint_video):
    visualize_joint_position(
        bvh_file=bvh_file + ".bvh",
        joint_name=joint_name,
        axis=axis,
        start_time=start_time,
        end_time=end_time,
        output_fps=output_fps,
        output_dir=output_dir,
        fig_size= (12, 4),  # Half height for joint visualization
        dpi= 200
    )

### Part 2: Combine the videos

In [5]:
def get_video_paths(output_dir, filename, joint_name="LeftAnkle", axis='y'):
    """Get dictionary mapping view names to their expected video paths"""
    return {
        'front': os.path.join(output_dir, "front_view.mp4"),
        'right': os.path.join(output_dir, "right_view.mp4"),
        # 'left': os.path.join(output_dir, "left_view.mp4"),
        # 'top': os.path.join(output_dir, "top_view.mp4"),
        
        'dundun': os.path.join(output_dir, "Dun.mp4"),
        'J1': os.path.join(output_dir, "J1.mp4"),
        'J2': os.path.join(output_dir, "J2.mp4"),
        'combined': os.path.join(output_dir, "drum_combined.mp4"),
        
        'joint_pos': os.path.join(output_dir, f"{joint_name}_{axis}_position.mp4"),
        'video_mix': os.path.join(output_dir, f"{filename}_pre_R_Mix_trimmed.mp4"),
        'audio': os.path.join(output_dir, f"{filename}_pre_R_Mix_trimmed_audio.mp3")
        
    }

In [ ]:
# Choose layout
layout = {
    'L1': [
        {'view': 'front', 'x': 0, 'y': 0, 'width': 320, 'height': 360},
        {'view': 'right', 'x': 320, 'y': 0, 'width': 320, 'height': 360},
        {'view': 'video_mix', 'x': 640, 'y': 0, 'width': 640, 'height': 360},
        {'view': 'combined', 'x': 0, 'y': 360, 'width': 1280, 'height': 360}
    ],
    'L2': [
        {'view': 'front', 'x': 0, 'y': 0, 'width': 640, 'height': 360},
        {'view': 'right', 'x': 640, 'y': 0, 'width': 640, 'height': 360},
        {'view': 'joint_position', 'x': 0, 'y': 360, 'width': 1280, 'height': 360}
    ]
    }   

canvas_size = (1280, 720)   
joint_name = "LeftAnkle"  
axis = 'y'

layout_config = layout['L1']

# Get video paths
view_videos = get_video_paths(output_dir, filename, joint_name, axis)

# Check if we have all required videos for the layout
required_views = {item['view'] for item in layout_config}
missing_views = required_views - set(view_videos.keys())

if missing_views:
    print(f"Warning: Missing videos for views: {missing_views}")
    print("Please generate the videos first")


# Combine the videos
combine_views(
    filename=bvh_file,
    start_time=start_time,
    end_time=end_time,
    output_dir=output_dir,
    view_videos=view_videos,
    layout_config=layout_config,
    video_size=canvas_size,
    fps=output_fps
    )

## Dance onset

In [8]:
import os
import numpy as np
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd

def load_cycles(cycles_csv_path):
    df = pd.read_csv(cycles_csv_path)
    return df["Virtual Onset"].values

def load_dance_onsets(dance_csv_path):
    df = pd.read_csv(dance_csv_path)
    return df["feet"].values

def find_cycle_phases(onsets, cycles):
    # For each onset, find which cycle it belongs to and compute its phase
    cycle_indices = np.searchsorted(cycles, onsets, side='right') - 1
    valid = (cycle_indices >= 0) & (cycle_indices < len(cycles) - 1)
    valid_onsets = onsets[valid]
    cycle_indices = cycle_indices[valid]
    phases = (valid_onsets - cycles[cycle_indices]) / (cycles[cycle_indices + 1] - cycles[cycle_indices])
    return cycle_indices, phases, valid_onsets

def kde_estimate(phases, SIG=0.01):
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(phases, bw_method=SIG)
    kde_xx = np.linspace(0, 1, 200)
    kde_h = kde(kde_xx)
    return kde_xx, kde_h

def analyze_dance_phases_no_plot(cycles_csv_path, dance_csv_path, W_start, W_end, SIG=0.01):
    # Load data
    cycles = load_cycles(cycles_csv_path)
    all_onsets = load_dance_onsets(dance_csv_path)
    # Filter onsets by time window
    window_mask = (all_onsets >= W_start) & (all_onsets <= W_end)
    onsets = all_onsets[window_mask]
    if len(cycles) < 2 or len(onsets) == 0:
        return None, None, None, None
    cycle_indices, phases, valid_onsets = find_cycle_phases(onsets, cycles)
    if len(phases) == 0:
        return None, None, None, None
    window_positions = (valid_onsets - W_start) / (W_end - W_start)
    kde_xx, kde_h = kde_estimate(phases, SIG)
    return phases, window_positions, kde_xx, kde_h

def animate_dance_phase_analysis(
    file_name, W_start, W_end, cycles_csv_path, dance_csv_path,
    figsize=(10, 3), dpi=100, save_dir=None
):
    """
    Animate the phase analysis plot for dance onsets with a moving playhead.
    """
    cycles = load_cycles(cycles_csv_path)
    phases, window_positions, kde_xx, kde_h = analyze_dance_phases_no_plot(
        cycles_csv_path, dance_csv_path, W_start, W_end
    )
    if phases is None:
        print("Not enough data for animation.")
        return

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    color = 'purple'
    kde_color = 'orange'

    # Plot scatter points (above zero)
    ax.scatter(phases, window_positions, alpha=0.5, color=color, s=5)

    # Scale KDE to be between -0.5 and 0, starting from bottom
    kde_scaled = -0.5 + (0.5 * kde_h / np.max(kde_h))
    ax.fill_between(kde_xx, -0.5, kde_scaled, alpha=0.3, color=kde_color)

    ax.set_xlabel('Normalized metric cycle')
    ax.set_ylabel('Relative Position in Window')
    ax.set_title(f'File: {file_name} | Window: {W_start:.1f}s - {W_end:.1f}s | Onset: Dance')

    ax.set_ylim(-0.55, 1.0)
    yticks = np.arange(0, 1.1, 0.2)
    ax.set_yticks(yticks)
    ax.grid(True, alpha=0.3)

    playhead, = ax.plot([0, 0], [-0.55, 1.0], 'k-', lw=1, alpha=0.7)
    h_playhead, = ax.plot([0, 1], [0, 0], 'k-', lw=1, alpha=0.7)

    def find_phase(t):
        idx = np.searchsorted(cycles, t)
        if idx == 0 or idx >= len(cycles):
            return None
        c = idx - 1
        L_c = cycles[c]
        L_c1 = cycles[c + 1]
        return (t - L_c) / (L_c1 - L_c)

    def update(frame):
        phase = find_phase(frame)
        if phase is not None:
            playhead.set_xdata([phase, phase])
            y_pos = (frame - W_start) / (W_end - W_start)
            h_playhead.set_ydata([y_pos, y_pos])
            ax.set_title(f'File: {file_name} | Window: {W_start:.1f}s - {W_end:.1f}s | Onset: Dance | Time: {frame:.2f}s')
        return playhead, h_playhead,

    frames = np.arange(W_start, W_end, 0.05)
    anim = animation.FuncAnimation(
        fig, update, frames=frames,
        interval=50, blit=True
    )
    plt.tight_layout()

    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        save_filename = f"dance.mp4"
        save_path = os.path.join(save_dir, save_filename)
        print(f"\nSaving animation to: {save_path}")
        try:
            writer = animation.FFMpegWriter(fps=24, bitrate=2000)
            anim.save(save_path, writer=writer)
            plt.close(fig)
            print("Animation saved successfully!")
        except Exception as e:
            print(f"Error saving animation: {str(e)}")
            plt.close(fig)
    else:
        print("Error: save_dir must be provided")
        plt.close(fig)
    return anim

if __name__ == "__main__":
    # Example usage
    file_name = "BKO_E1_D2_03_Suku"
    cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
    dance_csv_path = f"data/dance_onsets/{file_name}_T_dance_onsets.csv"
    W_start = 60.0
    W_end = 120.0
    figsize = (10, 3)
    dpi = 150
    save_dir = "phase_analysis_animations"

    animate_dance_phase_analysis(
        file_name, W_start, W_end,
        cycles_csv_path, dance_csv_path,
        figsize=figsize, dpi=dpi, save_dir=save_dir
    )


Saving animation to: phase_analysis_animations\dance.mp4
Animation saved successfully!


## Raw trajectory animation

In [4]:
import os
import numpy as np
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd

def animate_trajectories(
    file_name: str,
    W_start: float,
    W_end: float,
    base_path_logs: str = "data/logs_v1_may",
    frame_rate: float = 240,
    figsize: tuple = (10, 3),
    dpi: int = 100,
    save_dir: str = None
):
    """
    Animate the raw foot trajectories with a moving vertical playhead.
    """
    # Build file paths
    logs_onset_dir = os.path.join(base_path_logs, f"{file_name}_T", "onset_info")
    left_zpos_csv = os.path.join(logs_onset_dir, f"{file_name}_T_left_foot_zpos.csv")
    right_zpos_csv = os.path.join(logs_onset_dir, f"{file_name}_T_right_foot_zpos.csv")

    # Load data
    Lz = pd.read_csv(left_zpos_csv)["zpos"].values
    Rz = pd.read_csv(right_zpos_csv)["zpos"].values
    n_frames = len(Lz)
    times = np.arange(n_frames) / frame_rate

    # Trim to window
    win_mask = (times >= W_start) & (times <= W_end)
    t_win = times[win_mask]
    L_win = Lz[win_mask]
    R_win = Rz[win_mask]

    # Create figure and axis
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    
    # Plot trajectories
    ax.plot(t_win, L_win, '-', color='blue', alpha=0.5, label='Left Foot')
    ax.plot(t_win, R_win, '--', color='red', alpha=0.5, label='Right Foot')
    
    # Set y-axis limits based on the data
    y_min = min(L_win.min(), R_win.min())
    y_max = max(L_win.max(), R_win.max())
    y_range = y_max - y_min
    ax.set_ylim(y_min - 0.1*y_range, y_max + 0.1*y_range)
    
    # Create vertical playhead
    v_playhead, = ax.plot([W_start, W_start], [y_min - 0.1*y_range, y_max + 0.1*y_range], 
                         'k-', lw=2, alpha=0.7)
    
    # Set up the plot
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Foot Position')
    ax.set_title(f'File: {file_name} | Window: {W_start:.1f}s - {W_end:.1f}s')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper left')
    
    def update(frame):
        """Update function for animation."""
        # Update vertical playhead position
        v_playhead.set_xdata([frame, frame])
        # Update title with current time
        ax.set_title(f'File: {file_name} | Window: {W_start:.1f}s - {W_end:.1f}s | Time: {frame:.2f}s')
        return v_playhead,
    
    # Create animation
    print("\nCreating animation...")
    frames = np.arange(W_start, W_end, 0.05)  # 50ms steps
    print(f"Animation will have {len(frames)} frames")
    print(f"Time range: {frames[0]:.2f}s - {frames[-1]:.2f}s")
    
    anim = animation.FuncAnimation(
        fig, update, frames=frames,
        interval=50, blit=True
    )
    
    # Apply tight_layout before saving
    plt.tight_layout()
    
    if save_dir:
        # Create save directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)
        
        # Create filename
        save_filename = f"{file_name}_trajectories.mp4"
        save_path = os.path.join(save_dir, save_filename)
        
        print(f"\nSaving animation to: {save_path}")
        try:
            # Save animation as MP4
            writer = animation.FFMpegWriter(fps=24, bitrate=2000)
            anim.save(save_path, writer=writer)
            plt.close(fig)  # Explicitly close the figure
            print("Animation saved successfully!")
        except Exception as e:
            print(f"Error saving animation: {str(e)}")
            plt.close(fig)  # Close figure even if there's an error
    else:
        print("Error: save_dir must be provided")
        plt.close(fig)  # Close figure if no save directory
    
    return anim

if __name__ == "__main__":
    # Example usage
    file_name = "BKO_E1_D2_03_Suku"
    W_start = 60.0
    W_end = 80.0
    figsize = (10, 3)
    dpi = 150
    save_dir = "trajectory_animations"
    
    animate_trajectories(
        file_name, W_start, W_end,
        figsize=figsize, dpi=dpi,
        save_dir=save_dir
    )


Creating animation...
Animation will have 400 frames
Time range: 60.00s - 79.95s

Saving animation to: trajectory_animations\BKO_E1_D2_03_Suku_trajectories.mp4
Animation saved successfully!
